<a href="https://colab.research.google.com/github/mobassir94/comprehensive-bangla-tts/blob/main/TTS_text_preprocessing/in_depth_mlt_text_processing_for_bn_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install bnnumerizer==0.0.2
! pip install bangla==0.0.2


In [ ]:
import bangla
import re
from bnnumerizer import numerize

In [ ]:
#updated and modified from https://github.com/mobassir94/comprehensive-bangla-tts/blob/main/mlt_TTS_inference_demo/bn-ar-tts-pipeline.ipynb

attribution_dict={"সাঃ":"সাল্লাল্লাহু আলাইহি ওয়া সাল্লাম",                   
                                  "আঃ":"আলাইহিস সালাম",
                                  "রাঃ":"রাদিআল্লাহু আনহু",
                                  "রহঃ":"রহমাতুল্লাহি আলাইহি",
                                  "রহিঃ":"রহিমাহুল্লাহ",
                                  "হাফিঃ":"হাফিযাহুল্লাহ",
                                  "দাঃবাঃ":"দামাত বারাকাতুহুম,দামাত বারাকাতুল্লাহ"}

def tag_text(text):
    '''
        * tags arabic text with <ar>text</ar>
        * tags bangla text with <bn>text</bn>
    '''
    # remove multiple spaces
    text=re.sub(' +', ' ',text)
    # create start and end
    text="start"+text+"end"
    # tag text
    parts=re.split(u'[\u0600-\u06FF]+', text)
    # remove non chars
    parts=[p for p in parts if p.strip()]
    # unique parts
    parts=set(parts)
    # tag the text
    for m in parts:
        if len(m.strip())>1:text=text.replace(m,f"</ar><SPLIT><bn>{m}</bn><SPLIT><ar>")
    # clean-tags
    text=text.replace("</ar><SPLIT><bn>start",'<bn>')
    text=text.replace("end</bn><SPLIT><ar>",'</bn>')
    return text


def expand_full_attribution(text):
        '''
            expands full attribution
        '''
        for k,v in attribution_dict.items():text=text.replace(k,v)
        return text

def collapse_whitespace(text):
    # Regular expression matching whitespace:
    _whitespace_re = re.compile(r"\s+")
    return re.sub(_whitespace_re, " ", text)

def process_mlt_text(text):
    '''
        process multilingual text for suitable MLT TTS format
            * expand attributions
            * numerize text
            * tag sections of the text
            * sequentially list text blocks
            * Split based on sentence ending Characters

    '''
    # expand attributes
    text=expand_full_attribution(text)
    # english numbers to bangla conversion
    res = re.search('[0-9]', text)
    if res is not None:
      text = bangla.convert_english_digit_to_bangla_digit(text)
    # numerize text
    text=numerize(text)
    # tag sections
    text=tag_text(text)

    # text blocks
    blocks=text.split("<SPLIT>")
    blocks=[b for b in blocks if b.strip()]
    # create tuple of (lang,text)
    data=[]
    for block in blocks:
        lang=None
        if "<bn>" in block:
            block=block.replace("<bn>",'').replace("</bn>",'')
            lang="bn"
        elif "<ar>" in block:
            block=block.replace("<ar>",'').replace("</ar>",'')
            lang="ar"
    
        #Split based on sentence ending Characters
        if lang == "bn":
            bn_text = block.strip()

            sentenceEnders = re.compile('[।!?]')
            sentences = sentenceEnders.split(str(bn_text))

            for i in range(len(sentences)):
                res = re.sub('\n','',sentences[i])
                res = collapse_whitespace(res)
                res += '।'
                data.append((lang,res))

        elif lang == "ar":
            ar_text = block.strip()
            ar_text = re.sub("؟", "?", ar_text) # replace any ؟ with ?

            sentenceEnders = re.compile('[.!?]')
            sentences = sentenceEnders.split(str(ar_text))

            for i in range(len(sentences)):
                res = re.sub('\n','',sentences[i])
                res = collapse_whitespace(res)
                data.append((lang,res))
    return data


In [ ]:
text = '''[1] الحَمد এর মধ্যে যে ال রয়েছে, তা استغراق (সমূদয়) অথবা اختصاص (নির্দিষ্টীকরণ)এর অর্থে ব্যবহার হয়েছে। 
অর্থাৎ, সমস্ত প্রশংসা আল্লাহর জন্যই বা তাঁর জন্য নির্দিষ্ট; কেননা প্রশংসার প্রকৃত অধিকারী একমাত্র মহান আল্লাহই।
 কারো মধ্যে যদি কোন গুণ, সৌন্দর্য এবং কৃতিত্ব থাকে, তবে তাও মহান আল্লাহ কর্তৃক সৃষ্ট। অতএব প্রশংসার অধিকারী তিনিই। 
 ‘আল্লাহ’ শব্দটি মহান আল্লাহর সত্তার এমন এক সতন্ত্র নাম যার ব্যবহার অন্য কারো জন্য করা বৈধ নয়। 
 ‘আলহামদু লিল্লাহ’ কৃতজ্ঞতা-জ্ঞাপক বাক্য। এর বহু ফযীলতের কথা হাদীসসমূহে এসেছে। 
 একটি হাদীসে ‘লা-ইলাহা ইল্লাল্লা-হ’কে উত্তম জিকির বলা হয়েছে এবং ‘আলহামদু লিল্লাহ’কে উত্তম দুআ বলা হয়েছে। 
 (তিরমিযী, নাসায়ী ইত্যাদি) সহীহ মুসলিম এবং নাসায়ীর বর্ণনায় এসেছে, ‘আলহামদু লিল্লাহ’ দাঁড়িপাল্লা ভর্তি করে দেয়।
  এ জন্যই অন্য এক বর্ণনায় এসেছে যে, আল্লাহ এটা পছন্দ করেন যে, প্রত্যেক পানাহারের পর বান্দা তাঁর প্রশংসা করুক। 
  (সহীহ মুসলিম)[2] رَبّ মহান আল্লাহর সুন্দর নামসমূহের অন্যতম। যার অর্থ হল, প্রত্যেক জিনিসকে সৃষ্টি ক’রে তার প্রয়োজনীয় জিনিসের ব্যবস্থা ক’রে তাকে পরিপূর্ণতা দানকারী। 
  কোন জিনিসের প্রতি সম্বন্ধ (ইযাফত) না করে এর ব্যবহার অন্য কারো জন্য বৈধ নয়। عَالَمِيْن عَالَم (বিশ্ব-জাহান) শব্দের বহুবচন। তবে সকল সৃষ্টির সমষ্টিকে عَالَم বলা হয়। 
  এই জন্যেই এর বহুবচন ব্যবহার হয় না। কিন্তু এখানে তাঁর (আল্লাহর) পূর্ণ প্রতিপালকত্ব প্রকাশের জন্য এরও বহুবচন ব্যবহার করা হয়েছে। 
  এ থেকে উদ্দেশ্য হল, সৃষ্টির ভিন্ন ভিন্ন শ্রেণী বা সম্প্রদায়। যেমন, জ্বিন সম্প্রদায়, মানব সম্প্রদায়, ফিরিশ্তাকুল এবং জীব-জন্তু ও পশু-পক্ষীকুল ইত্যাদি। 
  এই সমস্ত সৃষ্টির প্রয়োজনসমূহও একে অপর থেকে অবশ্যই ভিন্নতর। কিন্তু বিশ্ব-প্রতিপালক প্রত্যেকের অবস্থা, 
  পরিস্থিতি এবং প্রকৃতি ও দেহ অনুযায়ী তার প্রয়োজনীয় জিনিসের ব্যবস্থা করে থাকেন।তাফসীরে আহসানুল বায়ান'''



In [ ]:
mlt_text = process_mlt_text(text)

In [ ]:
mlt_text

[('bn', '[এক]।'),
 ('ar', 'الحَمد'),
 ('bn', 'এর মধ্যে যে।'),
 ('ar', 'ال'),
 ('bn', 'রয়েছে, তা।'),
 ('ar', 'استغراق'),
 ('bn', '(সমূদয়) অথবা।'),
 ('ar', 'اختصاص'),
 ('bn', '(নির্দিষ্টীকরণ)এর অর্থে ব্যবহার হয়েছে।'),
 ('bn',
  ' অর্থাৎ, সমস্ত প্রশংসা আল্লাহর জন্যই বা তাঁর জন্য নির্দিষ্ট; কেননা প্রশংসার প্রকৃত অধিকারী একমাত্র মহান আল্লাহই।'),
 ('bn',
  ' কারো মধ্যে যদি কোন গুণ, সৌন্দর্য এবং কৃতিত্ব থাকে, তবে তাও মহান আল্লাহ কর্তৃক সৃষ্ট।'),
 ('bn', ' অতএব প্রশংসার অধিকারী তিনিই।'),
 ('bn',
  ' ‘আল্লাহ’ শব্দটি মহান আল্লাহর সত্তার এমন এক সতন্ত্র নাম যার ব্যবহার অন্য কারো জন্য করা বৈধ নয়।'),
 ('bn', ' ‘আলহামদু লিল্লাহ’ কৃতজ্ঞতা-জ্ঞাপক বাক্য।'),
 ('bn', ' এর বহু ফযীলতের কথা হাদীসসমূহে এসেছে।'),
 ('bn',
  ' একটি হাদীসে ‘লা-ইলাহা ইল্লাল্লা-হ’কে উত্তম জিকির বলা হয়েছে এবং ‘আলহামদু লিল্লাহ’কে উত্তম দুআ বলা হয়েছে।'),
 ('bn',
  ' (তিরমিযী, নাসায়ী ইত্যাদি) সহীহ মুসলিম এবং নাসায়ীর বর্ণনায় এসেছে, ‘আলহামদু লিল্লাহ’ দাঁড়িপাল্লা ভর্তি করে দেয়।'),
 ('bn',
  ' এ জন্যই অন্য এক বর্ণনায় এসেছে যে, আল্লাহ এটা প

In [ ]:
mlt_text[8][1]

'(নির্দিষ্টীকরণ)এর অর্থে ব্যবহার হয়েছে'